# 通过多个 CPU 优化推理

现在，某些模型在使用 PyTorch 时支持内置的张量并行（TP）。张量并行将模型拆分到多个 GPU 上，从而支持更大规模的模型，并行化诸如矩阵乘法等计算。

要启用张量并行，可以在调用 `from_pretrained()` 函数时传递参数 `tp_plan="auto"`：


In [ ]:
import os
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

# 初始化分布式环境
rank = int(os.environ["RANK"])  # 获取当前进程的排名
device = torch.device(f"cuda:{rank}")  # 选择对应的 GPU 设备
torch.distributed.init_process_group("nccl", device_id=device)  # 初始化分布式进程组

# 获取张量并行模型
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    tp_plan="auto",  # 启用自动张量并行
)

# 准备输入数据
tokenizer = AutoTokenizer.from_pretrained(model_id)  # 加载分词器
prompt = "Can I help"  # 输入提示
inputs = tokenizer(prompt, return_tensors="pt").input_ids.to(device)  # 将输入转换为张量并移动到 GPU

# 分布式运行
outputs = model(inputs)


你可以使用 `torchrun` 来启动上述脚本，每个进程映射到一个 GPU：


In [ ]:
torchrun --nproc-per-node 4 demo.py


PyTorch 张量并行目前支持以下模型：

* [Llama](https://huggingface.co/docs/transformers/model_doc/llama#transformers.LlamaModel)

如果你希望为其他模型添加张量并行支持，可以通过在 GitHub 上创建 Issue 或 Pull Request 来提出请求。

### 期待的加速效果

在推理时，你可以从显著的加速中受益，尤其是在处理大批量输入或长序列时。

对于 [Llama](https://huggingface.co/docs/transformers/model_doc/llama#transformers.LlamaModel) 模型，当序列长度为 512 时，不同批量大小下单次前向传播的预期加速效果如下图所示：

![](https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/transformers/Meta-Llama-3-8B-Instruct,%20seqlen%20=%20512,%20python,%20w_%20compile.png)